In [1]:
def __clear_env():
    for key in globals().keys():
        if not key.startswith("__"):# 排除系统内建函数
            globals().pop(key)
__clear_env
import example_pendulum
import torch; torch.manual_seed(66)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
#import torchvision
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
from torch.autograd import Variable

9.1177832613044e-28


In [2]:
file_route = R'C:\Users\87106\OneDrive\sindy\progress\AE'
device = 'cuda:0'
data = example_pendulum.get_pendulum_data(40)
val_data = example_pendulum.get_pendulum_data(10)
save_params = True
load_params = False

load_date = '4-13'
load_ver = 2
widths = [1024,512,128,64]
params={}
params['ver'] = "3"
params['accumulate_epochs'] = 0
params['date'] = '4-13'
params['widths'] = widths
params['activation'] = 'sigmoid'
params['max_epochs'] = 2000
params['epoch_size'] = data["x"].shape[0]
params['batch_size'] = 500
params['learning_rate'] = 0
params['loss_weight_x'] = 1
params['loss_weight_dx'] = 0
params['loss_weight_ddx'] = 0
params['learning_rate_switching_point'] = 500
loss_history = []
loss_z_history = []


In [3]:
params['learning_rate_stage1'] = 1e-4
params['learning_rate_stage2'] = 1e-4
params['learning_rate'] = params['learning_rate_stage1']

In [4]:
def autoencoder(input_dim,latent_dim,widths,device):
    #generate the parameters of the autoencoder
    encoder_weights,encoder_biases = build_network_layers(input_dim,latent_dim,widths,device)
    decoder_weights, decoder_biases = build_network_layers(latent_dim, input_dim, widths[::-1],device)
    return encoder_weights,encoder_biases,decoder_weights,decoder_biases


In [5]:
def build_network_layers(input_dim,output_dim,widths,device):
    #universal function for building network
    weights = []
    biases = []
    last_width = input_dim
    #middle layers
    for i,n_units in enumerate(widths):
        w = torch.Tensor(last_width,n_units,).to(device)
        nn.init.xavier_uniform_(w, gain=1.0)
        w = Variable(w, requires_grad=True)
        b = torch.Tensor(n_units).to(device)
        nn.init.constant_(b, 0.0)
        b = Variable(b, requires_grad=True)
        last_width = n_units
        weights.append(w)
        biases.append(b)
    #latent layer
    w = torch.Tensor(last_width,output_dim).to(device)
    nn.init.xavier_uniform_(w, gain=1.0)
    w = Variable(w,requires_grad=True)
    b = torch.Tensor(output_dim).to(device)
    nn.init.constant_(b, 0.0)
    b = Variable(b, requires_grad=True)
    weights.append(w)
    biases.append(b)
    return weights,biases

In [6]:
def autoencoder_forward(input,params):
    #pass through the autoencoder
    for i,weights in enumerate(params['encoder_weights']):
        input = torch.matmul(input,weights)+params['encoder_biases'][i]
        if params['activation'] == 'sigmoid' and i <len(params['encoder_weights'])-1:
            input = torch.sigmoid(input)
    for i,weights in enumerate(params['decoder_weights']):
        input = torch.matmul(input,weights)+params['decoder_biases'][i]
        if params['activation'] == 'sigmoid'and i <len(params['decoder_weights'])-1:
            input = torch.sigmoid(input)
    return input
def encoder_forward(input,params):
    for i,weights in enumerate(params['encoder_weights']):
        input = torch.matmul(input,weights)+params['encoder_biases'][i]
        if params['activation'] == 'sigmoid' and i <len(params['encoder_weights'])-1:
            input = torch.sigmoid(input)
    return input

def decoder_forward(input,params):
    for i,weights in enumerate(params['decoder_weights']):
        input = torch.matmul(input,weights)+params['decoder_biases'][i]
        if params['activation'] == 'sigmoid' and i <len(params['decoder_weights'])-1:
            input = torch.sigmoid(input)
    return input

In [7]:

def encoder_derivative(image,image_t,image_tt,params):
    #calculate the time/second time derivative of the latent(s) using the autoencoder
    dz = image_t
    ddz = image_tt
    input = image
    weights = params['encoder_weights']
    biases = params['encoder_biases']
    if params['activation'] == 'sigmoid':
        for i in range(len(weights)-1):
            input = torch.matmul(input,weights[i])+biases[i]
            input = torch.sigmoid(input)
            dz_prev = torch.matmul(dz,weights[i])
            sigmoid_derivative = torch.multiply(input,1-input)
            sigmoid_derivative2 = torch.multiply(sigmoid_derivative,1-2*input)
            dz = torch.multiply(sigmoid_derivative,dz_prev)
            ddz = torch.multiply(sigmoid_derivative2,torch.square(dz_prev))\
                  + torch.multiply(sigmoid_derivative,torch.matmul(ddz,weights[i]))
        dz = torch.matmul(dz,weights[-1])
        ddz = torch.matmul(ddz,weights[-1])
    return dz,ddz

def decoder_derivative(z,dz,ddz,params):
    #calculate the time/second time derivative of the latent(s) using the autoencoder
    input = z
    weights = params['decoder_weights']
    biases = params['decoder_biases']
    if params['activation'] == 'sigmoid':
        for i in range(len(weights)-1):
            input = torch.matmul(input,weights[i])+biases[i]
            input = torch.sigmoid(input)
            dz_prev = torch.matmul(dz,weights[i])
            sigmoid_derivative = torch.multiply(input,1-input)
            sigmoid_derivative2 = torch.multiply(sigmoid_derivative,1-2*input)
            dz = torch.multiply(sigmoid_derivative,dz_prev)
            #print(sigmoid_derivative.shape)
            ddz = torch.multiply(sigmoid_derivative2,torch.square(dz_prev))\
                  + torch.multiply(sigmoid_derivative,torch.matmul(ddz,weights[i]))
        dz = torch.matmul(dz,weights[-1])
        ddz = torch.matmul(ddz,weights[-1])
    return dz,ddz

In [8]:
def training(data,params,device):
    opt = torch.optim.Adam(generate_parameter(params), lr=params['learning_rate'])
    total_loss = 0
    total_loss_z = 0
    for j in range(params['epoch_size']//params['batch_size']):
        batch_idxs = np.arange(j * params['batch_size'], (j + 1) * params['batch_size'])
        z = torch.from_numpy(data['z'][batch_idxs]).to(torch.float32).to(device)
        x = torch.from_numpy(data['x'][batch_idxs]).to(torch.float32).to(device)
        dx = torch.from_numpy(data['dx'][batch_idxs]).to(torch.float32).to(device)
        ddx = torch.from_numpy(data['ddx'][batch_idxs]).to(torch.float32).to(device)
        x_predict = autoencoder_forward(x,params)
        z_predict = encoder_forward(x,params)
        dz_predict,ddz_predict = encoder_derivative(x,dx,ddx,params)
        dx_predict,ddx_predict = decoder_derivative(z_predict,dz_predict,ddz_predict,params)
        loss_x = torch.mean((x - x_predict)**2)
        loss_dx = torch.mean((dx - dx_predict) ** 2)
        loss_ddx = torch.mean((ddx - ddx_predict) ** 2)
        loss_z = torch.mean((z - z_predict)**2)
        loss = params['loss_weight_x'] * loss_x \
               + params['loss_weight_dx'] * loss_dx \
               + params['loss_weight_ddx'] * loss_ddx
        total_loss += loss
        total_loss_z += loss_z
        loss.backward()
        opt.step()
        opt.zero_grad()
    avg_loss = total_loss/(params['epoch_size']//params['batch_size'])
    avg_loss_z = total_loss_z/(params['epoch_size']//params['batch_size'])
    return avg_loss,avg_loss_z

In [9]:
def generate_parameter(params):
    encoder_weights = params['encoder_weights']
    encoder_biases = params['encoder_biases']
    params_temp = []
    for i in range(len(params['widths'])):
        params_temp.append(encoder_weights[i])
        params_temp.append(encoder_biases[i])
    decoder_weights = params['decoder_weights']
    decoder_biases = params['decoder_biases']
    for i in range(len(params['widths'])):
        params_temp.append(decoder_weights[i])
        params_temp.append(decoder_biases[i])
    return params_temp

def generate_vhat(vhat):
    vhat = tuple(vhat)
    yield vhat

In [10]:
def validation(data,params,device):
    total_loss = 0
    total_loss_z = 0
    params['epoch_size'] = val_data["x"].shape[0]
    for j in range(params['epoch_size']//params['batch_size']):
        batch_idxs = np.arange(j * params['batch_size'], (j + 1) * params['batch_size'])
        z = torch.from_numpy(data['z'][batch_idxs]).to(torch.float32).to(device)
        x = torch.from_numpy(data['x'][batch_idxs]).to(torch.float32).to(device)
        dx = torch.from_numpy(data['dx'][batch_idxs]).to(torch.float32).to(device)
        ddx = torch.from_numpy(data['ddx'][batch_idxs]).to(torch.float32).to(device)
        x_predict = autoencoder_forward(x,params)
        z_predict = encoder_forward(x,params)
        dz_predict,ddz_predict = encoder_derivative(x,dx,ddx,params)
        dx_predict,ddx_predict = decoder_derivative(z_predict,dz_predict,ddz_predict,params)
        loss_x = torch.mean((x - x_predict)**2)
        loss_dx = torch.mean((x - dx_predict) ** 2)
        loss_ddx = torch.mean((x - ddx_predict) ** 2)
        loss_z = torch.mean((z - z_predict)**2)
        loss = params['loss_weight_x'] * loss_x \
               + params['loss_weight_dx'] * loss_dx \
               + params['loss_weight_ddx'] * loss_ddx
        total_loss += loss
        total_loss_z += loss_z
        print('batch_{} --- image loss: {} --- z loss: {}'.format(j,loss,loss_z))
    avg_loss = total_loss/(params['epoch_size']//params['batch_size'])
    avg_loss_z = total_loss_z/(params['epoch_size']//params['batch_size'])
    return avg_loss,avg_loss_z

In [11]:
def saving(params):
    PATH = os.path.join('/mnt/ssd1/stilrmy/Autoencoder-conservtive_expression/progress',params['date'],params['ver'])
    if os.path.exists(PATH) == False:
        os.makedirs(PATH)
    params_names = [params['encoder_weights'],params['encoder_biases'],params['decoder_weights'],params['decoder_biases']]
    params_names_str = ['encoder_weights','encoder_biases','decoder_weights','decoder_biases']
    for j,param_set in enumerate(params_names):
        for i,elements in enumerate(param_set):
            np.save(r"/mnt/ssd1/stilrmy/Autoencoder-conservtive_expression/progress/{}/{}/autoencoder_{}_layer{}.npy".format(params['date'],params['ver'],params_names_str[j],i),
                   elements.clone().detach().cpu().numpy())
    sub_params = params
    del sub_params['encoder_weights']
    del sub_params['encoder_biases']
    del sub_params['decoder_weights']
    del sub_params['decoder_biases']
    with open(r"/mnt/ssd1/stilrmy/Autoencoder-conservtive_expression/progress/{}/{}/params.txt".format(params['date'],params['ver']),'w') as file:
        file.write(str(sub_params))
        file.write('\r\t')
        file.close()
    print('params saved')
    return

def loading_coef(params,date,load_number,file_route,device):
    print('loading params')
    params_names_str = ['encoder_weights', 'encoder_biases', 'decoder_weights', 'decoder_biases']
    for param_name in params_names_str:
        params['{}'.format(param_name)] = []
        for i in range(len(params['widths'])+1):
            file_name = 'autoencoder_{}_layer{}.npy'.format(param_name,i)
            route = os.path.join(file_route,date,load_number,file_name)
            temp_loader = np.load(route)
            temp_loader = torch.tensor(temp_loader).to(device)
            temp_loader = Variable(temp_loader, requires_grad=True)
            print(temp_loader.shape)
            params['{}'.format(param_name)].append(temp_loader)
    #coef = np.load(r'{}\{}\{}\coef.npy'.format(file_route,date,load_number))
    #coef = torch.Tensor(coef).to(device)
    #coef = Variable(coef,requires_grad=True)
    #expr = np.load(r'{}\{}\{}\expr.npy'.format(file_route,date,load_number))
    return params


In [12]:
if load_params == True:
    params = loading(params,load_date,load_ver,file_route,device)
else:
    encoder_weights,encoder_biases,decoder_weights,decoder_biases = autoencoder(2601,1,widths,device)
    params['encoder_weights'] = encoder_weights
    params['encoder_biases'] = encoder_biases
    params['decoder_weights'] = decoder_weights
    params['decoder_biases'] = decoder_biases

In [13]:
for epochs in range(params['max_epochs']):
    if epochs >= params['learning_rate_switching_point']:
        params['learning_rate'] = params['learning_rate_stage2']
    loss,loss_z = training(data,params,device)
    loss_history.append(loss.clone().detach().cpu().numpy())
    loss_z_history.append(loss_z.clone().detach().cpu().numpy())
    print('epoch: {} === loss: {}\t=== loss_z: {}'.format(epochs,loss,loss_z))

validation(val_data,params,device)


epoch: 0 === loss: 0.09892183542251587	=== loss_z: 2.465928316116333
epoch: 1 === loss: 0.03013359010219574	=== loss_z: 6.419044017791748
epoch: 2 === loss: 0.011420903727412224	=== loss_z: 9.549126625061035
epoch: 3 === loss: 0.008587858639657497	=== loss_z: 11.542319297790527
epoch: 4 === loss: 0.008241996169090271	=== loss_z: 12.546937942504883
epoch: 5 === loss: 0.008184676989912987	=== loss_z: 12.86254596710205
epoch: 6 === loss: 0.008170746266841888	=== loss_z: 12.949576377868652
epoch: 7 === loss: 0.00816622469574213	=== loss_z: 12.989842414855957
epoch: 8 === loss: 0.008164362981915474	=== loss_z: 13.014670372009277
epoch: 9 === loss: 0.008163432590663433	=== loss_z: 13.032136917114258
epoch: 10 === loss: 0.008162901736795902	=== loss_z: 13.045422554016113
epoch: 11 === loss: 0.008162564598023891	=== loss_z: 13.056048393249512
epoch: 12 === loss: 0.008162335492670536	=== loss_z: 13.064859390258789
epoch: 13 === loss: 0.008162173442542553	=== loss_z: 13.072361946105957
epoch: 14

(tensor(0.0082, device='cuda:0', grad_fn=<DivBackward0>),
 tensor(1.4951, device='cuda:0', grad_fn=<DivBackward0>))

In [14]:
if save_params == True:
    saving(params)
plotting(loss_history,loss_z_history)

params saved


NameError: name 'plotting' is not defined

In [ ]:
params['encoder_weights']

In [ ]:
params